# Testing Celery on Kubernetes with Science Abstracts
This is the big test.   we will first load the science abstracts and then use them in a test.  
## the test
Started a cluster on the google cloud.   We have a rabbitMQ server running on JetStream.  
The container being used is dbgannon/predictor.   
We will grab a subset of the data and push it to the the Queue.   The predictor will return the predicted topic classifications.   

In [1]:
import pickle
import json

In [2]:
def load_docs(path, name):
    filename = path+name+".p"
    fileobj = open(filename, "rb")
    z = fileobj.read()
    lst = pickle.loads(str(z))
    titles = []
    sitenames = []
    abstracts = []
    for i in range(0, len(lst)):
        titles.extend([lst[i][0]])
        sitenames.extend([lst[i][1]])
        abstracts.extend([lst[i][2]])
        
    print "done loading "+filename
    return abstracts, sitenames, titles

In [3]:
abstracts, sites, titles = load_docs("/Users/dennisgannon/Documents/sciml-data/OneDrive-2016-08-09/", 
                                     "sciml_data_arxiv")

done loading /Users/dennisgannon/Documents/sciml-data/OneDrive-2016-08-09/sciml_data_arxiv.p


In [4]:
print abstracts[1300]
print sites[1300]
print titles[1300]

Continuous-time random walks (CTRWs) on discrete state spaces, ranging from regular lattices to complex networks, are ubiquitous across physics, chemistry, and biology. Models with coarse-grained states, for example those in Markov models of molecular kinetics, or spatial disorder can give rise to memory and non-exponential distributions of waiting times and first-passage statistics. However, existing methods for analyzing CTRWs on complex energy landscapes do not address these effects. We therefore use statistical mechanics of the nonequilibrium path ensemble to characterize first-passage CTRWs on networks with arbitrary connectivity, energy landscape, and waiting time distributions. Our approach is valuable for calculating higher moments (beyond the mean) of path statistics such as path length, time, and action, as well as any conservative or nonconservative force along a path. For homogeneous networks we derive exact relations between these moments, quantifying the validity of appro

In [5]:
def read_config(subj, basepath):
    docpath =basepath+ "/config/config_"+subj+".json"
    with open(docpath, 'rb') as f:
        doc = f.read() 
    z =json.loads(doc)
    subject = z['subject']
    loadset = z['loadset']
    subtopics = []
    for w in z['supertopics']:
        subtopics.extend([(w[0], w[1])])
    return subject, loadset, subtopics


In [6]:
subject, loadset, subtopics = read_config("all4", "/Users/dennisgannon/Documents/sciml-data/OneDrive-2016-08-09")

In [7]:
print subject
#print loadset
#the loadset is the set of Arxiv subtopics of the subject we are looking for.
# the subtopics are the specific subdiciplines in the submect and the corresponding element of the load set in 
# which they are classified.
#subtopics[0]

all


In [8]:
def maintopcs():
    maindic = {}
    i = 0
    for x in subtopics:
        maindic[x[0]]= i
        i = i+1
    return maindic
    print subtopics[1][0]

In [9]:
maindic = maintopcs()

In [10]:
maindic

{u'Physics': 0, u'bio': 2, u'compsci': 3, u'finance': 4, u'math': 1}

In [11]:
def make_topic_dict(subtopics):
    dic  = {}
    for x in subtopics:
        maintop = x[0]
        subs = x[1]
        for s in subs:
            dic[s] = maintop
    return dic

In [12]:
topicdict = make_topic_dict(subtopics)

In [13]:
print len(abstracts)

7109


In [14]:
topicdict[sites[1300]]

u'Physics'

Now let's generate a train set and test set for facebooks classifier

In [15]:
import random
from random import randint

In [16]:
str(3)

'3'

In [21]:
ftrain = open('/users/dennisgannon/fastText/train.txt','w')
ftest =  open('/users/dennisgannon/fastText/test.txt','w')
for i in range(int(0.75*len(abstracts))):
    j = randint(0,len(abstracts)-1)
    if sites[j] != 'nlin.CG' and sites[j]!='none' and abstracts[j].find('$')< 0:
        strng = abstracts[j]
        k = strng.find('.')
        line = strng[0:k]
        bigtop =line[0:15]
        topic = topicdict[sites[j]]       
        label = str(maindic[topic])+' , '+ bigtop + ' , '+line
        ftrain.write(label+'.\n')           

In [18]:
j

2007

# Now initialize celery and check everything out

In [12]:
from celery import Celery
import time

In [13]:
app = Celery('predictor', broker='amqp://guest@%s//' % '149.165.157.221', backend='amqp')

In [14]:
@app.task
def predict_text(words, i):
    return ['local call']

In [15]:
@app.task
def print_wait_hello(t, id):
    return t 

In [16]:
x = print_wait_hello.apply_async([5, 4448])

In [17]:
x.get()

[4448]

In [18]:
y = predict_text.apply_async([abstracts[1310], 1310])

In [19]:
x = y.get()

In [20]:
x[0]

u'rf= compsci, best = compsci, nextb =compsci, cent =bio'

In [21]:
def score(s, correct):
    c1 = s.find("rf= ")
    l1 = s.find(",")
    s1 = s[c1+4:l1]
    c2 = s.find('best = ')
    l2 = s.find(', nextb')
    s2 = s[c2+7:l2]
    sc = 0
    if s1 == correct:
        sc = 1
    if s2 == correct:
        sc = 1
    return sc

In [22]:
score(x[0], topicdict[sites[1310]])

1

In [23]:
retvals = []
correct = []
count = 7000
offset = 0
for i in range(count):
    retvals.append(predict_text.apply_async([abstracts[i+offset], i]))
    correct.append(sites[i+offset])

In [25]:
t0 = time.time()
res = [retval.get() for retval in retvals]
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471216681.77
t1=1471219020.61
time elapsed = 2338.84190822


one pod

for 10  time elapsed = 4.42296910286

for 100 time elapsed = 62.9162750244

two pods

time elapsed = 51.7355389595

four pods

time elapsed = 35.1132261753

8 pods

time elapsed = 41.3670258522

16 pods

time elapsed = 43.4914228916

32 pods

time elapsed = 56.3568620682

In [26]:
res[0]

[u'rf= compsci, best = compsci, nextb =compsci, cent =compsci']

In [29]:
print len(correct)

7000


In [45]:
tot = 0.0
nones = 0
for i in range(len(correct)):
    goof = False
    try:
        top = topicdict[correct[i]]
    except:
        nones += 1
        goof = True
    if goof == False:
        tot = tot+score(res[i][0], top)
print "average score =" + str(tot/(len(correct)-nones))

average score =0.937982280652


In [46]:
nones

2

In [24]:
@app.task
def predict(i):
    return ['local call']

In [56]:
retvals = []
correct = []
count = 100
offset = 6000
for i in range(count):
    retvals.append(predict.apply_async([offset+ i]))
    correct.append(sites[i+6000])

In [57]:
t0 = time.time()
res = [retval.get() for retval in retvals]
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471369480.84
t1=1471369516.73
time elapsed = 35.8836040497


one pod
time elapsed = 68.2084679604
time elapsed = 67.6506841183
time elapsed = 67.5036990643

2 pods
time elapsed = 35.0938289165
time elapsed = 35.3820288181
time elapsed = 34.6607341766

4 pods
time elapsed = 34.8159649372
time elapsed = 33.6211850643
time elapsed = 34.3667662144

5 pods
time elapsed = 37.370057106
time elapsed = 34.6072402
time elapsed = 35.2972719669


six pods 
time elapsed = 36.8534381
time elapsed = 34.6813261509
time elapsed = 34.833958149

ten pods
time elapsed = 36.8795990944
time elapsed = 35.8836040497

In [58]:
retvals = []
count = 100
delay = 10
for i in range(count):
    retvals.append(print_wait_hello.apply_async([delay, i]))
 

In [59]:
t0 = time.time()
res = [retval.get() for retval in retvals]
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471476852.33
t1=1471477128.89
time elapsed = 276.562952042


1 pod
time elapsed = 995.476099014

1 surf pod
1006.0

1 single pod + 1
time elapsed = 522.981867075

2 pods
time elapsed = 494.070230007

2 single pods + 1
time elapsed = 358.69045186

3 single pods + 1
time elapsed = 276.562952042

4 pods
time elapsed = 242.368831873

4 single pods +1
time elapsed = 235.255094051

5 single pods +1
time elapsed = 196.179260969

6 single pods +1
time elapsed = 175.418498993

8 pods
time elapsed = 120.797874928 time elapsed = 121.223648071

16 pods
time elapsed = 99.2593250275 time elapsed = 60.8758580685 time elapsed = 61.1965789795

32 pods

time elapsed = 42.7353610992

In [77]:
totalwork = 100*10
speedup = totalwork/61.0

In [78]:
speedup

16.39344262295082

In [25]:
@app.task
def predict_many(i, count):
    return ['local call']

In [100]:
retvals = []
correct = []
count = 5
offset = 6000
for i in range(count):
    retvals.append(predict_many.apply_async([offset+ i, 120]))
    correct.append(sites[i+6000])

In [101]:
t0 = time.time()
res = [retval.get() for retval in retvals]
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471380442.69
t1=1471380585.48
time elapsed = 142.791425943


6 pods 60 tasks of 10 predicts each total work = 600 predicts 
time elapsed = 184.443963051
6 pods 30 tasks of 20 predicts each total work = 600 predicts 
time elapsed = 176.352622032
6 pods 15 tasks of 40 predicts each total work = 600 predicts
time elapsed = 146.799216032
6 pods 6 tasks of 100 predicts each total work = 600 predicts
time elapsed = 180.755970001
time elapsed = 179.902398825

1 pod 60 tasks of 10 predicts each total work = 600 predicts
time elapsed = 394.87173605
1 pod 30 tasks of 20 predicts each total work = 600 predicts
time elapsed = 387.621109962
1 pod 15 tasks of 40 predicts each total work = 600 predicts
time elapsed = 382.39035511
1 pods 6 tasks of 100 predicts each total work = 600 predicts
time elapsed = 373.496650934

2 pods 60 tasks of 10 predicts each total work = 600 predicts
time elapsed = 194.941056013
2 pods 30 tasks of 20 predicts each total work = 600 predicts
time elapsed = 188.393370152
2 pods 15 tasks of 40 predicts each total work = 600 predicts
time elapsed = 198.636164904
2 pods 6 tasks of 100 predicts each total work = 600 predicts
time elapsed = 181.407305002

5 pods 60 tasks of 10 predicts each total work = 600 predicts 
time elapsed = 144.982601166
5 pods 30 tasks of 20 predicts each total work = 600 predicts 
time elapsed = 145.315975904
5 pods 15 tasks of 40 predicts each total work = 600 predicts
time elapsed = 144.852874041
5 pods 5 tasks of 120 predicts each total work = 600 predicts
time elapsed = 142.791425943




In [63]:
res[0]

[u'predict from 6000: finance finance, bio bio, compsci compsci, math math, compsci compsci, Physics bio, Physics Physics, Physics Physics, compsci compsci, compsci Physics, bio bio, math math, Physics Physics, math math, Physics Physics, math compsci, Physics Physics, compsci bio, finance finance, Physics Physics, ']

In [26]:
@app.task
def predict_many_blank(i, count):
    return ['local call']

In [41]:
retvals = []
correct = []
count = 300
offset = 6000
for i in range(count):
    retvals.append(predict_many_blank.apply_async([offset+ i, 2]))
    correct.append(sites[i+6000])

In [42]:
t0 = time.time()
res = [retval.get() for retval in retvals]
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471469401.37
t1=1471469637.87
time elapsed = 236.495509148


5 pods 300 tasks of 2 predicts each total work = 600 predicts 
time elapsed = 127.197695017
5 pods 60 tasks of 10 predicts each total work = 600 predicts 
time elapsed = 136.39486599 vs. time elapsed = 144.982601166
5 pods 30 tasks of 20 predicts each total work = 600 predicts 
time elapsed = 149.096962929 vs, time elapsed = 145.315975904
5 pods 15 tasks of 40 predicts each total work = 600 predicts
time elapsed = 144.852874041
5 pods 5 tasks of 120 predicts each total work = 600 predicts
time elapsed = 142.791425943

5 small pods 5 tasks of 120 predicts each total work = 600 predicts
time elapsed = 177.240824938 time elapsed = 166.749773979

5 small pods 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 116.711094856 time elapsed = 118.18574214
1 small pods 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 410.411929846
1 small pods 60 tasks of 10 predicts each total work = 600 predicts 
time elapsed = 377.092034101
2 small pods 60 tasks of 10 predicts each total work = 600 predicts
time elapsed = 230.348160028 
5 small pods 60 tasks of 10 predicts each total work = 600 predicts
time elapsed = 152.949729919 time elapsed = 160.996865988

6 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 111.748270035 time elapsed = 111.397089005 
5 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 112.084007978
4 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 107.88885498
3 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 144.35948801
2 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 144.791702986
1 small pods+1 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 236.495509148
1 small pods 300 tasks of 2 predicts each total work = 600 predicts
time elapsed = 410.411929846


In [43]:
410/107.7

3.806870937790158

In [39]:
def count_servers(lis, name):
    i = 0
    for x in lis:
        if x[0].find(name) > 0:
            i = i+1
    return i

In [43]:
count_servers(res, 'mtest-job-jblf5')

42

In [27]:
from celery import group

In [28]:
retvals = []
calvals = []
count = 20
offset = 6000
for i in range(count):
    calvals.append([offset+ i, 20])
retvals = group(predict_many_blank.s(i[0],i[1]) for i in calvals).apply_async() 
#print calvals

In [29]:
t0 = time.time()
x = retvals.get()
t1 = time.time()
print "t0="+str(t0)
print "t1="+str(t1)
print "time elapsed = " + str(t1-t0)

t0=1471457953.51
t1=1471458065.06
time elapsed = 111.547049046


1 pod 20 tasks of 20 predicts time elapsed = 229.314230919
2 pod 20 tasks of 20 predicts time elapsed = 148.924943924
4 pod 20 tasks of 20 predicts time elapsed = 114.858204126 time elapsed = 191.920756817
5 pod 20 tasks of 20 predicts time elapsed = 154.280801058 time elapsed = 138.789224863 time elapsed = 111.547049046



In [30]:
x

[[u'hello from e4ace4fb079b i=6000'],
 [u'hello from mtest-job-fp3gg i=6001'],
 [u'hello from mtest-job-bh508 i=6002'],
 [u'hello from mtest-job-jblf5 i=6003'],
 [u'hello from mtest-job-sxhzb i=6004'],
 [u'hello from e4ace4fb079b i=6005'],
 [u'hello from mtest-job-fp3gg i=6006'],
 [u'hello from mtest-job-bh508 i=6007'],
 [u'hello from mtest-job-jblf5 i=6008'],
 [u'hello from mtest-job-sxhzb i=6009'],
 [u'hello from e4ace4fb079b i=6010'],
 [u'hello from mtest-job-fp3gg i=6011'],
 [u'hello from mtest-job-bh508 i=6012'],
 [u'hello from mtest-job-jblf5 i=6013'],
 [u'hello from mtest-job-sxhzb i=6014'],
 [u'hello from e4ace4fb079b i=6015'],
 [u'hello from mtest-job-fp3gg i=6016'],
 [u'hello from mtest-job-bh508 i=6017'],
 [u'hello from mtest-job-jblf5 i=6018'],
 [u'hello from mtest-job-sxhzb i=6019']]